## Links

https://python.langchain.com/docs/expression_language/cookbook/retrieval

## Required Imports

In [25]:
import os
from typing import List, Tuple
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores.chroma import Chroma
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.embeddings import OpenAIEmbeddings

from langchain.globals import set_verbose, set_debug

set_verbose(True)
set_debug(True)

OpenAPIKey = os.environ.get("OPENAI_API_KEY")


## Simple Test for a Simple Prompt and Feed to the Model

In [26]:
prompt = PromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI(api_key=OpenAPIKey)

conversationChain = prompt | model

conversationChain.invoke({"foo": "bears"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "foo": "bears"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] Entering Prompt run with input:
{
  "foo": "bears"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "tell me a joke about bears"
  }
}
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a joke about bears"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Why don't bears wear shoes?\n\nBecause they have bear feet!",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
     

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

## Simple Test to see How ChatPromptTemplate works

In [27]:
prompt = ChatPromptTemplate.from_messages([
                ("system", "You are a helpful AI bot. Your name is {name}."),
                ("human", "Hello, how are you doing?"),
                ("ai", "I'm doing well, thanks!"),
                ("human", "{user_input}"),
            ])

conversationChain = prompt | model

conversationChain.invoke({
    "name": "Bob",
    "user_input": "What is your name?"
})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "name": "Bob",
  "user_input": "What is your name?"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "name": "Bob",
  "user_input": "What is your name?"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are a helpful AI bot. Your name is Bob.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
  

AIMessage(content='My name is Bob. How can I assist you today?')

## Create Chat Prompts

In [28]:
conversationPrompt = PromptTemplate.from_template(
"""Given the following conversation and a follow up question, rephrase the follow up 
question to be a standalone question, in its original language.

Chat History:
{chat_history}

------------------------------
Follow Up Input: {question}
Standalone question:""")

# Answer Prompt is the one that we expect the assistant to provide an answer in
answerPrompt = ChatPromptTemplate.from_template("""{context}

                                          
-----------
Answer the question below based only on the above context (without mention the context in 
the response).

Question: {question}
""")

## Create Conversational Chain & Invoke

In [29]:

def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer

# chain = RunnableParallel(
#     standalone_question=RunnablePassthrough.assign(
#         chat_history=lambda x: _format_chat_history(x["chat_history"])
#     )
# )
#   | conversationPrompt | ChatOpenAI(temperature=0) | StrOutputParser(),

model = ChatOpenAI(temperature=0,api_key=OpenAPIKey)
conversationChain = RunnableParallel(
    question={
    "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    "question": lambda x: x["question"]
    } | conversationPrompt | model | StrOutputParser()
)

conversationChain.invoke({
    "chat_history": [],
    "question": "Given that the universe is vast, I struggle to grasp the context of the meaning of life?"
})

[chain/start] [1:chain:RunnableParallel] Entering Chain run with input:
{
  "chat_history": [],
  "question": "Given that the universe is vast, I struggle to grasp the context of the meaning of life?"
}
[chain/start] [1:chain:RunnableParallel > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "chat_history": [],
  "question": "Given that the universe is vast, I struggle to grasp the context of the meaning of life?"
}
[chain/start] [1:chain:RunnableParallel > 2:chain:RunnableSequence > 3:chain:RunnableParallel] Entering Chain run with input:
{
  "chat_history": [],
  "question": "Given that the universe is vast, I struggle to grasp the context of the meaning of life?"
}
[chain/start] [1:chain:RunnableParallel > 2:chain:RunnableSequence > 3:chain:RunnableParallel > 4:chain:<lambda>] Entering Chain run with input:
{
  "chat_history": [],
  "question": "Given that the universe is vast, I struggle to grasp the context of the meaning of life?"
}
[chain/end] [1:chain:RunnableParal

{'question': 'What is the context of the meaning of life, considering the vastness of the universe?'}

## Use In-Memory Embeddings

If using FAISS
1. ```import sys```
2. ```!{sys.executable} -m pip install faiss-cpu```


In [30]:
loader = DirectoryLoader('./data/training', glob="**/*.md", loader_cls=TextLoader) # TextLoader == raw text
docs = loader.load()

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
    ("#", "Header 1"), # Head level, metadata key
    ("##", "Header 2"),
    ("###", "Header 3"),
])

splitted_docs: List[Document] = []

for doc in docs:     
  splitted_docs.extend(splitter.split_text(doc.page_content))

   
chroma = Chroma.from_documents(
  documents=splitted_docs,
  embedding=OpenAIEmbeddings(api_key=OpenAPIKey)
)

result = chroma.similarity_search("How do I install indoor sensors?")

for res in result:
  print(res)

page_content='How do I install the indoor climate sensors?  \nIt is important that you install the indoor sensors in the right place. The sensors should be placed in the middle of the room, at a height of 1.5 meters. The sensors should not be placed in direct sunlight or near heat sources such as radiators or lamps. Do not place any sensors near windows or doors.  \nPlace one sensor at the north wall and one at the south wall to be able to measure the temperature difference between the two sensors.' metadata={'Header 1': 'Crossbreed Smarter Heating', 'Header 2': 'Installation and Positioning of Indoor Climate Sensors'}
page_content='How do I install the indoor climate sensors?  \nIt is important that you install the indoor sensors in the right place. The sensors should be placed in the middle of the room, at a height of 1.5 meters. The sensors should not be placed in direct sunlight or near heat sources such as radiators or lamps. Do not place any sensors near windows or doors.  \nPlac

## Combine Question & Answer Chain with Chat History

In [33]:
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


answerChain = conversationChain | {
  "question": RunnablePassthrough(),
  "context": itemgetter("question") | chroma.as_retriever() | format_docs,
} | answerPrompt | model

answerChain.invoke({
    "chat_history": [],
    "question": "How do I install indoor sensors?"
})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "chat_history": [],
  "question": "How do I install indoor sensors?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "chat_history": [],
  "question": "How do I install indoor sensors?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "chat_history": [],
  "question": "How do I install indoor sensors?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 4:chain:RunnableParallel] Entering Chain run with input:
{
  "chat_history": [],
  "question": "How do I install indoor sensors?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 4:chain:RunnableParallel > 5:chain:<lambda>] Entering Chain run with input:
{
  "chat_history": [],
  "question": "How do I install indoor sensors?"
}
[ch

ChatPromptValue(messages=[HumanMessage(content='How do I install the indoor climate sensors?  \nIt is important that you install the indoor sensors in the right place. The sensors should be placed in the middle of the room, at a height of 1.5 meters. The sensors should not be placed in direct sunlight or near heat sources such as radiators or lamps. Do not place any sensors near windows or doors.  \nPlace one sensor at the north wall and one at the south wall to be able to measure the temperature difference between the two sensors.\n\nHow do I install the indoor climate sensors?  \nIt is important that you install the indoor sensors in the right place. The sensors should be placed in the middle of the room, at a height of 1.5 meters. The sensors should not be placed in direct sunlight or near heat sources such as radiators or lamps. Do not place any sensors near windows or doors.  \nPlace one sensor at the north wall and one at the south wall to be able to measure the temperature diffe